# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**: Luis Adrian Bravo Ramirez

**Professor**: Pablo Camarillo Ramirez

In [32]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://4f3c6067fdf1:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [33]:
!pwd

/opt/spark/work-dir


# Creating Agencies Dataframe

In [34]:
from pyspark.sql.functions import get_json_object 
from luisbravor00.spark_utils import SparkUtils

schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])
df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")
df_agencies = df_agencies.withColumn("agency_name", get_json_object(df_agencies.agency_info, '$.agency_name'))
df_agencies.show(truncate=False)


+---------+-----------------------------------------------------+-------------+
|agency_id|agency_info                                          |agency_name  |
+---------+-----------------------------------------------------+-------------+
|1        |{'agency_name': 'NYC Rentals', 'city': 'New York'}   |NYC Rentals  |
|2        |{'agency_name': 'LA Car Rental', 'city': 'Londres'}  |LA Car Rental|
|3        |{'agency_name': 'Zapopan Auto', 'city': 'Zapopan'}   |Zapopan Auto |
|4        |{'agency_name': 'SF Cars', 'city': 'San Francisco'}  |SF Cars      |
|5        |{'agency_name': 'Mexico Cars', 'city': 'Mexico City'}|Mexico Cars  |
+---------+-----------------------------------------------------+-------------+



# Creating Brands Dataframe

In [35]:
schema_brands = SparkUtils.generate_schema([("brand_id", "int"), ("brand_info", "string")])
df_brands = spark.read.schema(schema_brands).option("header", True).csv("/opt/spark/work-dir/data/car_service/brands")
df_brands.show(truncate=False)

+--------+-----------------------------------------------------+
|brand_id|brand_info                                           |
+--------+-----------------------------------------------------+
|1       |{'brand_name': 'Mercedes-Benz', 'country': 'Germany'}|
|2       |{'brand_name': 'BMW', 'country': 'Germany'}          |
|3       |{'brand_name': 'Audi', 'country': 'Germany'}         |
|4       |{'brand_name': 'Ford', 'country': 'US'}              |
|5       |{'brand_name': 'BYD', 'country': 'China'}            |
|6       |{'brand_name': 'Honda', 'country': 'Japan'}          |
|7       |{'brand_name': 'Toyota', 'country': 'Japan'}         |
+--------+-----------------------------------------------------+



# Creating Cars Dataframe

In [36]:
schema_cars = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])
df_cars = spark.read.schema(schema_cars).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")
df_cars = df_cars.withColumn("car_name", get_json_object(df_cars.car_info, '$.car_name'))
df_cars.show(truncate=False)

+------+----------------------------------------------------------------------------------------+------------------------------------+
|car_id|car_info                                                                                |car_name                            |
+------+----------------------------------------------------------------------------------------+------------------------------------+
|1     |{'car_name': 'Chang-Fisher Model 7', 'brand_id': 5, 'price_per_day': 139}               |Chang-Fisher Model 7                |
|2     |{'car_name': 'Sheppard-Tucker Model 4', 'brand_id': 6, 'price_per_day': 70}             |Sheppard-Tucker Model 4             |
|3     |{'car_name': 'Faulkner-Howard Model 5', 'brand_id': 3, 'price_per_day': 53}             |Faulkner-Howard Model 5             |
|4     |{'car_name': 'Wagner LLC Model 1', 'brand_id': 5, 'price_per_day': 89}                  |Wagner LLC Model 1                  |
|5     |{'car_name': 'Campos PLC Model 4', 'brand_id': 

# Creating Customers Dataframe

In [37]:
schema_customers = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])
df_customers = spark.read.schema(schema_customers).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers")
df_customers = df_customers.withColumn("customer_name", get_json_object(df_customers.customer_info, '$.customer_name'))
df_customers.show(truncate=False)

+-----------+-------------------------------------------------------------------------+--------------------+
|customer_id|customer_info                                                            |customer_name       |
+-----------+-------------------------------------------------------------------------+--------------------+
|1          |{'customer_name': 'Tiffany Riley', 'city': 'Monterrey', 'age': 32}       |Tiffany Riley       |
|2          |{'customer_name': 'Matthew Davies', 'city': 'Monterrey', 'age': 36}      |Matthew Davies      |
|3          |{'customer_name': 'Rebecca Miller', 'city': 'Mexico City', 'age': 30}    |Rebecca Miller      |
|4          |{'customer_name': 'Katelyn Mccoy', 'city': 'New York', 'age': 34}        |Katelyn Mccoy       |
|5          |{'customer_name': 'Dana Dennis', 'city': 'Zapopan', 'age': 26}           |Dana Dennis         |
|6          |{'customer_name': 'Daniel Norton', 'city': 'Mexico City', 'age': 34}     |Daniel Norton       |
|7          |{'cust

# Creating Rentals Dataframe

In [38]:
schema_rentals = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])
df_rentals = spark.read.schema(schema_rentals).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")
df_rentals = df_rentals \
    .withColumn("car_id", get_json_object(df_rentals.rental_info, '$.car_id')) \
    .withColumn("customer_id", get_json_object(df_rentals.rental_info, '$.customer_id')) \
    .withColumn("agency_id", get_json_object(df_rentals.rental_info, '$.agency_id'))
df_rentals.show(truncate=False)

+---------+--------------------------------------------------+------+-----------+---------+
|rental_id|rental_info                                       |car_id|customer_id|agency_id|
+---------+--------------------------------------------------+------+-----------+---------+
|11891    |{'car_id': 21, 'customer_id': 71, 'agency_id': 1} |21    |71         |1        |
|11892    |{'car_id': 11, 'customer_id': 52, 'agency_id': 2} |11    |52         |2        |
|11893    |{'car_id': 22, 'customer_id': 116, 'agency_id': 4}|22    |116        |4        |
|11894    |{'car_id': 5, 'customer_id': 107, 'agency_id': 1} |5     |107        |1        |
|11895    |{'car_id': 4, 'customer_id': 53, 'agency_id': 4}  |4     |53         |4        |
|11896    |{'car_id': 8, 'customer_id': 131, 'agency_id': 2} |8     |131        |2        |
|11897    |{'car_id': 23, 'customer_id': 66, 'agency_id': 3} |23    |66         |3        |
|11898    |{'car_id': 24, 'customer_id': 60, 'agency_id': 4} |24    |60         

# Obtaining final Dataframe


In [43]:
# Join all the dataframes
df_final = df_rentals \
    .join(df_cars, on='car_id', how='left') \
    .join(df_agencies, on='agency_id', how='left') \
    .join(df_customers, on='customer_id', how='left') \

# Filter out the values (not needed)

# Select the required columns
df_final = df_final.select("rental_id", "car_name", "agency_name", "customer_name")

df_final.show(truncate=False)

+---------+-----------------------------------+-------------+---------------+
|rental_id|car_name                           |agency_name  |customer_name  |
+---------+-----------------------------------+-------------+---------------+
|11891    |Wallace-Carlson Model 9            |NYC Rentals  |Margaret Jones |
|11892    |Grimes-Green Model 8               |LA Car Rental|Albert Williams|
|11893    |Stewart-Allen Model 5              |SF Cars      |Caleb Fleming  |
|11894    |Campos PLC Model 4                 |NYC Rentals  |Andrew Butler  |
|11895    |Wagner LLC Model 1                 |SF Cars      |Kristin Potts  |
|11896    |Jones, Jefferson and Rivera Model 7|LA Car Rental|Jeremy Parks   |
|11897    |Lopez and Sons Model 9             |Zapopan Auto |Terry Wells    |
|11898    |Salazar Ltd Model 8                |SF Cars      |Marc Williams  |
|11899    |Villanueva PLC Model 7             |LA Car Rental|Danny Williams |
|11900    |Faulkner-Howard Model 5            |SF Cars      |Eri

In [44]:
sc.stop()